In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import copy
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import SimpleITK as stk
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from skimage import measure

In [ ]:
import numpy as np
import os
import random
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
lung_names = os.listdir("/kaggle/input/luna16dataset/LUNA16/lungs_roi/")
random.shuffle(lung_names)

In [ ]:
n = len(lung_names)
s = int(n*.8)
train_lung_names = lung_names[:s]
test_lung_names = lung_names[s:]
print(len(train_lung_names))
print(len(test_lung_names))

In [ ]:
trainX = []
trainY = []

for lname in tqdm(train_lung_names):
  mname = lname.replace("lungs","masks")
  lung = np.load("/kaggle/input/luna16dataset/LUNA16/lungs_roi/"+lname)
  mask = np.load("/kaggle/input/luna16dataset/LUNA16/nodule_mask/"+mname)
  trainX.append(lung)
  trainY.append(mask)

trainX = np.array(trainX, dtype=np.uint8)
trainY = np.array(trainY, dtype=np.uint8)

np.save("trainX.npy", trainX)
np.save("trainY.npy", trainY)

In [ ]:
testX = []
testY = []

for lname in tqdm(test_lung_names):
  mname = lname.replace("lungs","masks")
  lung = np.load("/kaggle/input/luna16dataset/LUNA16/lungs_roi/"+lname)
  mask = np.load("/kaggle/input/luna16dataset/LUNA16/nodule_mask/"+mname)
  testX.append(lung)
  testY.append(mask)

testX = np.array(testX, dtype=np.uint8)
testY = np.array(testY, dtype=np.uint8)

np.save("testX.npy", testX)
np.save("testY.npy", testY)

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import SimpleITK as sitk 
from tqdm import tqdm
import numpy as np
from tensorflow import losses

from tensorflow.keras.utils import to_categorical,normalize
from tensorflow.keras.models import Model
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D,GlobalMaxPooling2D ,MaxPooling2D, UpSampling2D, Reshape, Dropout, GlobalAveragePooling2D, ConvLSTM2D, Conv2DTranspose, Dense, Multiply, BatchNormalization, Activation 
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.initializers as initializers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [ ]:
trainX = np.load("/kaggle/working/trainX.npy").astype(np.float32)
trainY = np.load("/kaggle/working/trainY.npy").astype(np.float32)
testX = np.load("/kaggle/working/testX.npy").astype(np.float32)
testY = np.load("/kaggle/working/testY.npy").astype(np.float32)

In [ ]:
trainX = (trainX-127.0)/127.0
testX = (testX-127.0)/127.0
trainY = (trainY>127).astype(np.float32)
testY = (testY>127).astype(np.float32)

In [ ]:
trainX = np.reshape(trainX, (len(trainX), 512, 512, 1))
trainY = np.reshape(trainY, (len(trainY), 512, 512, 1))
testX = np.reshape(testX, (len(testX), 512, 512, 1))
testY = np.reshape(testY, (len(testY), 512, 512, 1))

print("trainX:",trainX.shape)
print("trainY:",trainY.shape)
print("testX:",testX.shape)
print("testY:",testY.shape)

In [ ]:
# def calculate_iou(y_true, y_pred):
#     results = []
#     for i in range(0,y_true.shape[0]):
#         # set the types so we are sure what type we are using
#         y_true = y_true.astype(np.float32)
#         y_pred = y_pred.astype(np.float32)


#         # boxTrue
#         x_boxTrue_tleft = y_true[0,0]  # numpy index selection
#         y_boxTrue_tleft = y_true[0,1]
#         boxTrue_width = y_true[0,2]
#         boxTrue_height = y_true[0,3]
#         area_boxTrue = (boxTrue_width * boxTrue_height)

#         # boxPred
#         x_boxPred_tleft = y_pred[0,0]
#         y_boxPred_tleft = y_pred[0,1]
#         boxPred_width = y_pred[0,2]
#         boxPred_height = y_pred[0,3]
#         area_boxPred = (boxPred_width * boxPred_height)
#         # calculate the bottom right coordinates for boxTrue and boxPred

#         # boxTrue
#         x_boxTrue_br = x_boxTrue_tleft + boxTrue_width
#         y_boxTrue_br = y_boxTrue_tleft + boxTrue_height # Version 2 revision

#         # boxPred
#         x_boxPred_br = x_boxPred_tleft + boxPred_width
#         y_boxPred_br = y_boxPred_tleft + boxPred_height # Version 2 revision


#         # calculate the top left and bottom right coordinates for the intersection box, boxInt

#         # boxInt - top left coords
#         x_boxInt_tleft = np.max([x_boxTrue_tleft,x_boxPred_tleft])
#         y_boxInt_tleft = np.max([y_boxTrue_tleft,y_boxPred_tleft]) # Version 2 revision

#         # boxInt - bottom right coords
#         x_boxInt_br = np.min([x_boxTrue_br,x_boxPred_br])
#         y_boxInt_br = np.min([y_boxTrue_br,y_boxPred_br]) 
#         # Calculate the area of boxInt, i.e. the area of the intersection 
#         # between boxTrue and boxPred.
#         # The np.max() function forces the intersection area to 0 if the boxes don't overlap.
        
        
#         # Version 2 revision
#         area_of_intersection = \
#         np.max([0,(x_boxInt_br - x_boxInt_tleft)]) * np.max([0,(y_boxInt_br - y_boxInt_tleft)])

#         iou = area_of_intersection / ((area_boxTrue + area_boxPred) - area_of_intersection)


#         # This must match the type used in py_func
#         iou = iou.astype(np.float32)
        
#         # append the result to a list at the end of each loop
#         results.append(iou)
    
#     # return the mean IoU score for the batch
#     return np.mean(results)

In [ ]:
from keras import backend as K
from sklearn.metrics import jaccard_score

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)
def IoU(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    mask_sum=K.sum(y_true_f) + K.sum(y_pred_f)
    return (intersection+1)/(mask_sum-intersection+1)
# def iou_coef(y_true, y_pred, smooth=1):
#   intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
#   union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
#   iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
#   return iou
# def IoU(y_true, y_pred):
#     iou = tf.numpy_function(calculate_iou, [y_true, y_pred], tf.float32)
#     return iou

In [ ]:
#Get a handle to the attached TPU. On GCP it will be the CloudTPU itself
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
def SqueezeExcite(x, ratio=8):
    nb_chan = K.int_shape(x)[-1]

    y = GlobalAveragePooling2D()(x)
    y = Dense(nb_chan // ratio, activation='gelu')(y)
    y = Dense(nb_chan, activation='sigmoid')(y)

    y = Multiply()([x, y])
    return y


def dense_unet(input_size=(512,512,1)):
    inputs = Input(input_size)
    K.set_image_data_format('channels_last')
    dropout_rate = 0.4
    conv11 = Conv2D(32, (3, 3), activation='gelu', padding='same')(inputs)
    conc11 = concatenate([inputs, conv11], axis=3)
    conv12 = Conv2D(32, (3, 3), activation='gelu', padding='same')(conc11)
    conc12 = concatenate([inputs, conv12], axis=3)
    conc13=Dense(32)(conc12)
    drop1 = Dropout(rate=dropout_rate)(conc13, training=True)
    pool1 = MaxPooling2D(pool_size=(2, 2))(drop1)

    conv21 = Conv2D(64, (3, 3), activation='gelu', padding='same')(pool1)
    conc21 = concatenate([pool1, conv21], axis=3)
    conv22 = Conv2D(64, (3, 3), activation='gelu', padding='same')(conc21)
    conc22 = concatenate([pool1, conv22], axis=3)
    conc23= Dense(64)(conc22)
    drop2 = Dropout(rate=dropout_rate)(conc23, training=True)
    pool2 = MaxPooling2D(pool_size=(2, 2))(drop2)

    conv31 = Conv2D(128, (3, 3), activation='gelu', padding='same')(pool2)
    conc31 = concatenate([pool2, conv31], axis=3)
    conv32 = Conv2D(128, (3, 3), activation='gelu', padding='same')(conc31)
    conc32 = concatenate([pool2, conv32], axis=3)
    conc33= Dense(128)(conc32)
    drop3 = Dropout(rate=dropout_rate)(conc33, training=True)
    pool3 = MaxPooling2D(pool_size=(2, 2))(drop3)

    conv41 = Conv2D(256, (3, 3), activation='gelu', padding='same')(pool3)
    conc41 = concatenate([pool3, conv41], axis=3)
    conv42 = Conv2D(256, (3, 3), activation='gelu', padding='same')(conc41)
    conc42 = concatenate([pool3, conv42], axis=3)
    conc43= Dense(256)(conc42)
    drop4 = Dropout(rate=dropout_rate)(conc43, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv51 = Conv2D(512, (3, 3), activation='gelu', padding='same')(pool4)
    conc51 = concatenate([pool4, conv51], axis=3)
    conv52 = Conv2D(512, (3, 3), activation='gelu', padding='same')(conc51)
    conc52 = concatenate([pool4, conv52], axis=3)
    conc53= Dense(512)(conc52)
    drop5 = Dropout(rate=dropout_rate)(conc53, training=True)

    # Decoding Path of the ResUNet
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop5), conc42], axis=3)
    up6 = SqueezeExcite(up6, ratio=16)
    conv61 = Conv2D(256, (3, 3), activation='gelu', padding='same')(up6)
    conc61 = concatenate([up6, conv61], axis=3)
    conv62 = Conv2D(256, (3, 3), activation='gelu', padding='same')(conc61)
    conc62 = concatenate([up6, conv62], axis=3)
    drop6 = Dropout(rate=dropout_rate)(conc62, training=True)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(drop6), conv32], axis=3)
    up7 = SqueezeExcite(up7, ratio=16)
    conv71 = Conv2D(128, (3, 3), activation='gelu', padding='same')(up7)
    conc71 = concatenate([up7, conv71], axis=3)
    conv72 = Conv2D(128, (3, 3), activation='gelu', padding='same')(conc71)
    conc72 = concatenate([up7, conv72], axis=3)
    drop7 = Dropout(rate=dropout_rate)(conc72, training=True)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(drop7), conv22], axis=3)
    up8 = SqueezeExcite(up8, ratio=16)
    conv81 = Conv2D(64, (3, 3), activation='gelu', padding='same')(up8)
    conc81 = concatenate([up8, conv81], axis=3)
    conv82 = Conv2D(64, (3, 3), activation='gelu', padding='same')(conc81)
    conc82 = concatenate([up8, conv82], axis=3)
    drop8 = Dropout(rate=dropout_rate)(conc82, training=True)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(drop8), conv12], axis=3)
    up9 = SqueezeExcite(up9, ratio=16)
    conv91 = Conv2D(32, (3, 3), activation='gelu', padding='same')(up9)
    conc91 = concatenate([up9, conv91], axis=3)
    conv92 = Conv2D(32, (3, 3), activation='gelu', padding='same')(conc91)
    conc92 = concatenate([up9, conv92], axis=3)
    drop9 = Dropout(rate=dropout_rate)(conc92, training=True)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(drop9)

    model = Model(inputs, outputs)

    return model


In [ ]:
with strategy.scope():
  model = dense_unet(input_size = (512, 512, 1))
  model.compile(optimizer=Adam(learning_rate=2e-4),
              loss=[dice_coef_loss],
              metrics = [dice_coef, 'binary_accuracy', IoU])
model.summary()

In [ ]:
weight_path="checkpoint-{epoch:03d}-{val_dice_coef:.3f}.hdf5"
weight_path3="checkpoint3-{epoch:03d}-{val_dice_coef:.3f}.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
checkpoint3 = ModelCheckpoint(weight_path3, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.7, 
                                   patience=2, 
                                   verbose=1, mode='min', min_delta=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=18)

CALLBACKS = [checkpoint]
CALLBACKS3 = [checkpoint3]

In [ ]:
history = model.fit(x = trainX,
                    y = trainY,
                    batch_size = 32,
                    epochs = 1000,
                    validation_data =(testX,testY),
                    callbacks=CALLBACKS
                  )

In [ ]:
plt.figure(figsize=(20,6))
for i, met in enumerate(['loss', 'IoU']):
  plt.subplot(1,2,i+1)
  plt.plot(history.history[met], color="b")
  plt.plot(history.history["val_"+met], color="g")
  plt.title('Model '+met.capitalize())
  plt.xlabel('epochs')
  plt.ylabel(met)
  plt.legend(['train', 'val'])

plt.savefig('history_plot.png')

In [ ]:
plt.figure(figsize=(20,6))
# plt.subplot(1,2,i+1)
met='IoU'
plt.plot(history.history[met], color="b")
plt.plot(history.history["val_"+met], color="g")
plt.title('Model '+met.capitalize())
plt.xlabel('epochs')
plt.ylabel(met)
plt.legend(['train', 'val'])

plt.savefig('history_plot.png')

In [ ]:
model.evaluate(testX, testY)

In [ ]:
with strategy.scope():
  model.load_weights("/kaggle/working/checkpoint-643-0.862.hdf5")

  model.compile(optimizer=Adam(learning_rate=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model.evaluate(testX, testY)

In [ ]:
with strategy.scope():
  model.load_weights("/kaggle/working/checkpoint-643-0.862.hdf5")

  model.compile(optimizer=Adam(learning_rate=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model.evaluate(testX, testY)

In [ ]:
with strategy.scope():
  model2 = dense_unet(input_size=(512,512,1))
  model2.load_weights("/kaggle/working/checkpoint-643-0.862.hdf5")

  model2.compile(optimizer=Adam(learning_rate=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model2.evaluate(testX, testY)

In [ ]:
with strategy.scope():
  model2.load_weights("/kaggle/working/checkpoint-643-0.862.hdf5")

  model2.compile(optimizer=Adam(learning_rate=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model2.evaluate(testX, testY)

In [ ]:
with strategy.scope():
  model2.load_weights("/kaggle/working/checkpoint-643-0.862.hdf5")

  model2.compile(optimizer=Adam(learning_rate=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model2.evaluate(testX, testY)

In [ ]:
history2 = model2.fit(x = trainX,
                    y = trainY,
                    batch_size = 32,
                    epochs = 200,
                    validation_data =(testX,testY),
                    callbacks=CALLBACKS
                  )

In [ ]:
history = model3.fit(x = trainX,
                    y = trainY,
                    batch_size = 32,
                    epochs = 500,
                    validation_data =(testX,testY),
                    callbacks=CALLBACKS3
                  )

In [ ]:
model3.evaluate(testX, testY)

In [ ]:

with strategy.scope():
  model4 = dense_unet(input_size=(512,512,1))
  model4.load_weights("/kaggle/working/checkpoint3-431-0.842.hdf5")

  model4.compile(optimizer=tf.keras.optimizers.AdamW(lr=2e-4),
                loss=[dice_coef_loss],
                metrics = [dice_coef, 'binary_accuracy'])

model4.evaluate(testX, testY)

In [ ]:
plt.figure(figsize=(20,6))
for i, met in enumerate(['dice_coef', 'loss']):
  plt.subplot(1,2,i+1)
  plt.plot(history.history[met], color="b")
  plt.plot(history.history["val_"+met], color="g")
  plt.title('Model '+met.capitalize())
  plt.xlabel('epochs')
  plt.ylabel(met)
  plt.legend(['train', 'val'])

In [ ]:
def display(true, pred, X, m=0, n=50):
  n = len(list(true[m:n]))
  r = n//2 if n%2==0 else (n//2)+1
  plt.figure(figsize=(20,5*r))
  for i,(t, p, x) in enumerate(zip(true[m:n],pred[m:n],X[m:n])):
    t = np.squeeze(t)
    p = np.squeeze(p)
    x = np.squeeze(x)
    groundtruth = cv2.addWeighted(x,.5,t,.5,0)
    prediction = cv2.addWeighted(x,.5,p,.5,0)
    plt.subplot(r,2,i+1)
    plt.title("GroundTruth"+" "*36+"Prediction")
    ig = np.hstack((groundtruth, prediction))
    plt.imshow(ig, cmap="bone")

In [ ]:
pred = model3.predict(testX)

In [ ]:
display(testY,pred,testX)

In [ ]:
import copy
m = copy.deepcopy(np.squeeze(pred[46]))
plt.imshow(m, cmap="gray")

In [ ]:
m = copy.deepcopy(np.squeeze(testX[46]))
plt.imshow(m, cmap="gray")

In [ ]:
m = copy.deepcopy(np.squeeze(testY[46]))
plt.imshow(m, cmap="gray")

In [ ]:
import cv2
m2 = m.copy()
m2[10:12,10:20] = 1
plt.imshow(m2, cmap="gray")

In [ ]:
from skimage import measure
m[m>=0.5] = 255
m[m<0.5] = 0
m = m.astype(np.uint8)
labels = measure.label(m)
regions = measure.regionprops(labels)
     

In [ ]:
bb, cc, dd = [], [], []
for prop in regions:
  B = prop.bbox
  C = prop.centroid
  D = prop.equivalent_diameter
  bb.append((( max(0, B[1]-8), max(0, B[0]-8) ),( min(B[3]+8, 512), min(B[2]+8, 512) )))    # ((x1,y1),(x2,y2))
  cc.append(C)    # (y,x)
  dd.append(D)
     

In [ ]:

m2 = m.copy()
for rect in bb:
  m2 = cv2.rectangle(m2, rect[0], rect[1], (255), 2)
plt.imshow(m2, cmap="gray")

In [ ]:
m = copy.deepcopy(np.squeeze(pred[30]))
plt.imshow(m, cmap="gray")

In [ ]:
m = copy.deepcopy(np.squeeze(testX[30]))
plt.imshow(m, cmap="gray")

In [ ]:
m = copy.deepcopy(np.squeeze(testY[30]))
plt.imshow(m, cmap="gray")

In [ ]:
pred3 = model4.predict(testX)

m = copy.deepcopy(np.squeeze(pred3[46]))
plt.imshow(m, cmap="gray")

In [ ]:
display(testY,pred3,testX)